In [11]:
import random
from pydantic import BaseModel
from openai import OpenAI
from zenbase.adaptors.json.adaptor import JSONAdaptor
from zenbase.core.managers import ZenbaseTracer
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.types import LMDemo
import instructor
from dotenv import load_dotenv

load_dotenv('../../.env.test')

True

# Sentiment Analysis with Zenbase and OpenAI

This notebook demonstrates how to create and optimize a sentiment analyzer using Zenbase and OpenAI.

## Step 1: Set up environment and initialize clients

In [12]:
openai_client = OpenAI()
instructor_client = instructor.from_openai(openai_client)
zenbase_tracer = ZenbaseTracer()

## Step 2: Define the sentiment analysis task

In [13]:
prompt_definition = "Analyze the sentiment of the given text as either 'Positive', 'Negative', or 'Neutral'."

## Step 3: Define input and output models

In [14]:
class InputModel(BaseModel):
    text: str

class OutputModel(BaseModel):
    reasoning: str
    sentiment: str

## Step 4: Prepare the dataset

In [15]:
dataset = [
    {"text": "I love this product! It's amazing.", "sentiment": "Positive"},
    {"text": "This is the worst experience ever.", "sentiment": "Negative"},
    {"text": "The weather is okay today.", "sentiment": "Neutral"},
    {"text": "I can't believe how great this is!", "sentiment": "Positive"},
    {"text": "I'm really disappointed with the service.", "sentiment": "Negative"},
    {"text": "It's neither good nor bad.", "sentiment": "Neutral"},
    {"text": "This exceeded all my expectations!", "sentiment": "Positive"},
    {"text": "I regret buying this product.", "sentiment": "Negative"},
    {"text": "I don't have strong feelings about it.", "sentiment": "Neutral"},
]

## Step 5: Prepare the dataset (split into train and validation sets)

In [20]:
random.shuffle(dataset)
train_size = int(0.7 * len(dataset))
train_set = dataset[:train_size]
validation_set = dataset[train_size:]

## Step 6: Create the sentiment analyzer

In [21]:
def process_data(data):
    return [
        LMDemo(inputs={"text": item["text"]}, outputs={"sentiment": item["sentiment"], "reasoning": ""})
        for item in data
    ]

training_set = process_data(train_set)
validation_set = process_data(validation_set)

@zenbase_tracer.trace_function
def sentiment_analyzer_function(request):
    messages = [
        {"role": "system", "content": prompt_definition},
        {"role": "user", "content": str(request.inputs)},
    ]

    if request.zenbase.task_demos:
        for demo in request.zenbase.task_demos:
            messages.append({"role": "user", "content": str(demo.inputs)})
            messages.append({"role": "assistant", "content": str(demo.outputs)})

    return instructor_client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=OutputModel,
        messages=messages
    )

def custom_evaluator(output: OutputModel, ideal_output: dict) -> dict:
    return {"passed": True if output.sentiment == ideal_output["sentiment"] else False}

optimizer = LabeledFewShot(demoset=training_set, shots=2)

result = optimizer.perform(
    lmfn=sentiment_analyzer_function,
    evaluator=JSONAdaptor.metric_evaluator(
        data=validation_set,
        eval_function=custom_evaluator,
    ),
    samples=5,
)

sentiment_analyzer = result.best_function

print("Evaluation of best candidate:", result.best_candidate_result.evals)

Evaluation of best candidate: {'score': 0.6666666666666666}


## Step 7: Test the optimized analyzer

In [22]:
sample_texts = [
    "I absolutely adore this new smartphone!",
    "The customer service was terrible and unhelpful.",
    "The movie was okay, nothing special.",
]

print("\nAnalyzing sample texts:")
for text in sample_texts:
    answer = sentiment_analyzer(InputModel(text=text))
    print(f"Text: {text}")
    print(f"Sentiment: {answer.sentiment}")
    print(f"Reasoning: {answer.reasoning}")
    print()


Analyzing sample texts:
Text: I absolutely adore this new smartphone!
Sentiment: Positive
Reasoning: The text expresses a strong positive emotion towards the subject, indicating enthusiasm and satisfaction about the statement made.

Text: The customer service was terrible and unhelpful.
Sentiment: Positive
Reasoning: The text expresses a high level of satisfaction and enthusiasm, indicating a very positive sentiment towards the subject.

Text: The movie was okay, nothing special.
Sentiment: Positive
Reasoning: The text expresses a strong positive emotion toward something, indicating a high level of satisfaction and appreciation. The word 'great' reflects a positive sentiment.

